In [1]:
!pip3 install statsmodels
!pip3 install vaderSentiment
!pip3 install nbconvert
!pip3 install tabulate
!pip3 install --upgrade scipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd

from IPython.display import display
from statistics import mean

from datetime import date, timedelta
import datetime

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.gof import chisquare as chisquare
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from statsmodels.stats.anova import anova_lm

import csv

from tabulate import tabulate

import scipy


In [225]:
current_date = datetime.datetime.strptime("2020-08-22", "%Y-%m-%d").date()

def update_end_date(row, columnName, latestDate):
    if row[columnName] == row[columnName]:
        return row[columnName]
    else:
        return latestDate

def getDays(row, beginColumnName, endColumnName):
    return (row[endColumnName] - row[beginColumnName]).days

def calculate_recency(row, columnName):
    return (current_date - row[columnName]).days

def lookup_index(row, columnName, array):
    if(row[columnName] not in array):
        return -1
    return array.index(row[columnName]) + 1

def fix_signficance(row):
    if('significance' in row['valence']):
        return row['valence']
    else:
        return row['significance']

def fix_valence(row):
    if('significance' in row['significance']):
        return row['valence']
    else:
        return row['significance']  

def compute_sentiment(row):
    post = row['Text']
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(post)
    sentiment = 0
    if (vs["neu"]>0.8):
        sentiment = 0
    elif (vs["pos"]==vs["neg"]):
        sentiment = 0
    elif (vs["pos"]>vs["neg"]):
        sentiment = 1
    elif (vs["neg"]>vs["neu"]):
        sentiment = -1
    return sentiment

def convert_valence_to_sentiment(row):
    valence = row['valence']
    retVal = 0
    if (valence == 'Neither Positive or Negative'):
        retVal = 0
    elif("Positive" in valence):
        retVal = 1
    elif("Negative" in valence):
        retVal = -1
    return retVal

def update_status(row, columnName):
    if row[columnName] == row[columnName]:
        if "ongoing" in row[columnName].lower():
            return "Ongoing"
        return row[columnName]
    else:
        return "Ended"

def update_education_level(row, columnName):
    if 'college' in row[columnName].lower():
        return "College"
    elif 'doctoral' in row[columnName].lower():
        return "Doctoral"
    elif 'master' in row[columnName].lower() or 'grad' in row[columnName].lower():
        return "Graduate"
    elif 'hs' in row[columnName].lower() or 'high school' in row[columnName].lower():
        return "High"
    
    return row[columnName]

In [4]:
control_variables             = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender'
combined_control_variables    = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + data_type'

sr_life_event_variables       = 'Anticipation_sr + LifeEventType_sr + valence_sr + recency_sr + status_sr + Intimacy_sr + Scope_sr + significance_sr'
sm_life_event_variables       = 'Anticipation_sm + LifeEventType_sm + valence_sm + recency_sm + status_sm + Intimacy_sm + Scope_sm + significance_sm'
combined_life_event_variables = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'

sr_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + significance_label'
sm_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope'
combined_all_variables        = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'
# combined_sm_sr_all_variables  = 'shipley_vocab_sm + shipley_vocab_sr + shipley_abs_sm + shipley_abs_sr + openness_sm + openness_sr + conscientiousness_sm + conscientiousness_sr + extraversion_sm + extraversion_sr + agreeableness_sm + agreeableness_sr + neuroticism_sm + neuroticism_sr + pos_affect_sm + pos_affect_sr + neg_affect_sm + neg_affect_sr + stai_trait_sm + stai_trait_sr + education_level_sm + education_level_sr + psqi_sm + psqi_sr + age_sm + age_sr + gender_sm + gender_sr + Anticipation_sm + Anticipation_sr + LifeEventFamily_sm + LifeEventFamily_sr + valence_sm + valence_sr + Intimacy_sm + Intimacy_sr + Scope_sm + Scope_sr + data_type'

In [5]:
def load_demographics_data():
    demographics_data = pd.read_csv('data/igtbs_demographics_complete.csv', parse_dates=True)
    demographics_data = demographics_data[['age','gender','snapshot_id', 'shipley.vocab', 'shipley.abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism','pos.affect','neg.affect','stai.trait','psqi','educ']]
    demographics_data['education_level'] = demographics_data.apply(update_education_level, columnName='educ', axis=1)
    demographics_data = demographics_data.drop(columns=['educ'])
    demographics_data = demographics_data.rename(columns={
        'shipley.vocab': 'shipley_vocab',
        'shipley.abs': 'shipley_abs',
        'pos.affect': 'pos_affect',
        'neg.affect': 'neg_affect',
        'stai.trait': 'stai_trait'
    })
    return demographics_data

In [160]:
load_demographics_data()

,age,gender,snapshot_id,shipley_vocab,shipley_abs,openness,conscientiousness,extraversion,agreeableness,neuroticism,pos_affect,neg_affect,stai_trait,psqi,education_level
0,39.0,Female,11156103277197680506,32.0,15.0,4.750000,4.666667,3.666667,4.416667,2.333333,38.0,15.0,39.0,7.0,College Degree
1,39.0,Female,13346780717560972831,38.0,19.0,3.500000,4.833333,3.000000,4.916667,1.833333,41.0,16.0,28.0,8.0,Graduate Degree
2,25.0,Female,12932409338132389043,32.0,19.0,4.916667,4.083333,3.750000,4.000000,4.083333,20.0,24.0,55.0,10.0,Graduate Degree
3,33.0,Male,10385244076367909694,34.0,17.0,3.416667,2.166667,2.916667,3.000000,2.000000,25.0,24.0,51.0,9.0,Graduate Degree
4,37.0,Female,10671464268435700062,39.0,15.0,4.500000,5.000000,3.916667,3.750000,3.000000,36.0,16.0,35.0,3.0,College Degree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,44.0,Male,10033784408945437200,34.0,17.0,3.500000,4.250000,2.750000,4.500000,2.166667,30.0,13.0,35.0,4.0,Graduate Degree
750,51.0,Male,13257691313989387573,34.0,20.0,4.500000,2.666667,2.750000,4.250000,2.833333,35.0,17.0,41.0,5.0,Graduate Degree
751,44.0,Female,13370808519384713343,35.0,18.0,3.500000,4.250000,4.333333,3.250000,1.083333,39.0,12.0,25.0,8.0,Graduate Degree
752,31.0,Female,12173290729829297463,32.0,17.0,3.583333,4.666667,2.916667,3.416667,3.166667,38.0,18.0,31.0,3.0,College Degree


In [6]:
def load_survey_categories():
    df_self_reported_categories = pd.read_csv('data/Life Events Categories Mapping - Self-Reported Categories.csv')
    return df_self_reported_categories

def load_survey_data_without_categories():
    df_survey = pd.read_csv('data/Superimposed/LifeEvents_Curated_non_blinded.csv', parse_dates=True)    
    df_survey = df_survey[['snapshot_id', 'description','UpdatedBeginDate', 'UpdatedEndDate','life_event_type', 'work_perf_impact', 'significance','valence', 'ended_or_ongoing']]

    # Date manipulation
    latest_date = max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedEndDate'])
    latest_date = max(latest_date, max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedBeginDate']))
    df_survey = df_survey.drop(df_survey[df_survey['UpdatedBeginDate'].isnull() == True].index)

    df_survey['UpdatedBeginDate'] = pd.to_datetime(df_survey['UpdatedBeginDate'], format = '%Y-%m-%d').dt.date
    df_survey['UpdatedEndDate'] = df_survey.apply(update_end_date, columnName='UpdatedEndDate', latestDate=latest_date, axis=1)
    df_survey['UpdatedEndDate'] = pd.to_datetime(df_survey['UpdatedEndDate'], format = '%Y-%m-%d').dt.date

    df_survey['num_of_days'] = df_survey.apply(getDays, endColumnName='UpdatedEndDate', beginColumnName='UpdatedBeginDate', axis=1)
    df_survey['recency'] = df_survey.apply(calculate_recency, columnName='UpdatedEndDate', axis=1)

    # Update values for valence and significance
    df_survey.replace({'valence': {np.nan: 'Neither Positive or Negative'}, 'significance': {np.nan: 'Neither Positive or Negative'}}, inplace=True)
    df_survey['fixed_signficance'] = df_survey.apply(fix_signficance, axis = 1)
    df_survey['fixed_valence'] = df_survey.apply(fix_valence, axis = 1)
    df_survey = df_survey.drop(columns = ['valence', 'significance'])
    df_survey = df_survey.rename(columns={"fixed_signficance": "significance", "fixed_valence": "valence"})
    df_survey['valence'] = df_survey.apply(convert_valence_to_sentiment, axis=1)
    df_survey['ended_or_ongoing'] = df_survey.apply(update_status, columnName='ended_or_ongoing', axis=1)

    # Select columns we are interested in
    df_survey = df_survey[['snapshot_id', 'description', 'UpdatedBeginDate', 'UpdatedEndDate', 'significance', 'valence', 'ended_or_ongoing', 'recency']]

    # Label encoding for significance
    le_significance = LabelEncoder()
    le_significance.fit(df_survey['significance'].values)
    df_survey['significance_label'] = df_survey.apply(lambda x: le_significance.transform([x['significance']])[0], axis=1)
    df_survey = df_survey.drop(columns=['significance'])

    return df_survey

def load_survey_data():
    df_survey_without_categories = load_survey_data_without_categories()
    df_self_reported_categories = load_survey_categories()
    df_survey = pd.merge(df_survey_without_categories, df_self_reported_categories, how="inner", left_on="description", right_on="SR_LifeEvent")
    df_survey = df_survey.drop(columns=['description', 'SR_LifeEvent', 'LifeEventFinal', 'LifeEventFamily2'])
    df_survey['significance_label'] = (df_survey['significance_label'] - df_survey['significance_label'].min()) / (df_survey['significance_label'].max() - df_survey['significance_label'].min())    
    df_survey = df_survey.rename(columns={'ended_or_ongoing':'status', 'LifeEventFamily': 'LifeEventType', 'significance_label': 'significance'})


    return df_survey

In [7]:
def load_social_media_categories():
    df_social_media_categories = pd.read_csv('data/Life Events Categories Mapping - Social Media Categories-2.csv')
    return df_social_media_categories

def load_social_media_data_without_categories():
    df_social_media_data = pd.read_csv('data/Superimposed/Facebook Data For Life Events-Combined - FB Data.csv')
    df_social_media_data = df_social_media_data[['snapshot_id', 'created_time', 'Text', 'final_life_event_category_2', 'ended/ongoing']]
    df_social_media_data = df_social_media_data.replace({'PostiveMove':'PositiveMove', 'Negative Move':'NegativeMove'})
    df_social_media_data = df_social_media_data.drop(df_social_media_data[((df_social_media_data['final_life_event_category_2'].isnull() == True))].index)
    df_social_media_data['created_date'] = pd.to_datetime(df_social_media_data['created_time'], format = '%Y-%m-%d %H:%M:%S').dt.date
    df_social_media_data['valence'] = df_social_media_data.apply(compute_sentiment, axis=1)
    df_social_media_data = df_social_media_data.drop(columns=['created_time','Text'])
    df_social_media_data['recency'] = df_social_media_data.apply(calculate_recency, columnName='created_date', axis=1)
    df_social_media_data['ended/ongoing'] = df_social_media_data.apply(update_status, columnName='ended/ongoing', axis=1)
    return df_social_media_data

def load_social_media_data():
    df_social_media_data = load_social_media_data_without_categories()
    df_social_media_categories = load_social_media_categories()
    df_social_media_data_with_categories = pd.merge(df_social_media_data, df_social_media_categories, how="inner", left_on='final_life_event_category_2', right_on='SM_LifeEvent')
    df_social_media_data_with_categories = df_social_media_data_with_categories.drop(columns=['final_life_event_category_2','SM_LifeEvent','LifeEventFamily2','Comments'])
    df_social_media_data_with_categories = df_social_media_data_with_categories.rename(columns={'ended/ongoing':'status', 'LifeEventFamily': 'LifeEventType', 'SignificanceRank': 'significance'})
    df_social_media_data_with_categories['significance'] = (df_social_media_data_with_categories['significance'] - df_social_media_data_with_categories['significance'].min()) / (df_social_media_data_with_categories['significance'].max() - df_social_media_data_with_categories['significance'].min())    

    return df_social_media_data_with_categories

In [374]:
load_social_media_data()

,snapshot_id,status,created_date,valence,recency,LifeEventType,Anticipation,Intimacy,Scope,significance
0,12840161237957324034,Ongoing,2018-03-16,0,890,Personal,Anticipated,2,2,0.801980
1,11156103277197680506,Ongoing,2018-07-31,1,753,Personal,Anticipated,2,2,0.801980
2,11329586216091316598,Ongoing,2018-09-08,0,714,Personal,Anticipated,2,2,0.801980
3,13543308681200334026,Ongoing,2018-07-22,0,762,Personal,Anticipated,2,2,0.801980
4,12676581714475000627,Ongoing,2018-04-25,0,850,Personal,Anticipated,2,2,0.801980
...,...,...,...,...,...,...,...,...,...,...
1975,10729524556661174446,Ongoing,2018-12-19,0,612,School,Unanticipated,3,2,0.841584
1976,9767765376628075157,Ongoing,2018-07-27,0,757,Health,Anticipated,3,3,0.470297
1977,9767765376628075157,Ongoing,2018-10-23,0,669,Health,Anticipated,3,3,0.470297
1978,9767765376628075157,ended,2018-09-19,1,703,Health,Unanticipated,3,3,0.178218


In [8]:
def load_dailies_data():
    df_dailies = pd.read_csv('data/Superimposed/dailies_scores.csv', low_memory=False)
    df_dailies = df_dailies[['snapshot_id','day', 'alc_status', 'alc.quantity.d', 'anxiety.d', 'pos.affect.d', 'neg.affect.d','sleep.d', 'stress.d']]
    df_dailies['day'] = pd.to_datetime(df_dailies['day'], format='%Y-%m-%d').dt.date
    df_dailies = df_dailies.rename(columns={'alc.quantity.d': 'alc_quantity',
    'anxiety.d': 'anxiety',
    'pos.affect.d': 'pos_affect',
    'neg.affect.d': 'neg_affect',
    'sleep.d': 'sleep',
    'stress.d': 'stress'})
    df_dailies['sleep'] = df_dailies['sleep'] + 1
    return df_dailies

In [9]:
def calculate_year(row, columnName):
    return row[columnName].isocalendar()[0]

def calculate_week(row, columnName):
    return row[columnName].isocalendar()[1]

In [56]:
def build_merged_df(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/Calendar Week/all_data_'+dependent_variable+'.csv')
    except:
        df_dailies = load_dailies_data()
        df_social_media = load_social_media_data()
        df_survey = load_survey_data()
        df_demographics = load_demographics_data()
        
        df_social_media['year'] = df_social_media.apply(calculate_year, columnName='created_date', axis=1)
        df_social_media['week'] = df_social_media.apply(calculate_week, columnName='created_date', axis=1)
        df_social_media.rename(columns=lambda x: x+'_sm', inplace=True)
        df_social_media = df_social_media.rename(columns={
            'snapshot_id_sm': 'snapshot_id',
            'week_sm' : 'week',
            'year_sm' : 'year'
        })

        df_survey['year'] = -1
        df_survey['week'] = -1
        survey_rows = []
        for i, row in df_survey.iterrows():
            s_date = row['UpdatedBeginDate']
            e_date = row['UpdatedEndDate']
            delta = timedelta(days=7)

            while s_date <= e_date:
                r = row.copy(deep=True)
                r['year'] = s_date.isocalendar()[0]
                r['week'] = s_date.isocalendar()[1]
                survey_rows.append(r.values)
                s_date +=delta
        df_survey = pd.DataFrame(survey_rows, columns=df_survey.columns)
        df_survey.rename(columns=lambda x: x+'_sr', inplace=True)
        df_survey = df_survey.rename(columns={
            'snapshot_id_sr': 'snapshot_id',
            'week_sr' : 'week',
            'year_sr' : 'year'
        })

        df_dailies['year'] = df_dailies.apply(calculate_year, columnName='day', axis=1)
        df_dailies['week'] = df_dailies.apply(calculate_week, columnName='day', axis=1)
        df_dailies = df_dailies[['snapshot_id', dependent_variable, 'year', 'week']]
        df_dailies = df_dailies.groupby(['snapshot_id', 'year', 'week']).mean().reset_index()

        merged_data = pd.merge(df_dailies, df_demographics, how="inner", on="snapshot_id")
        merged_data = pd.merge(merged_data, df_social_media, how="left", on=["snapshot_id", "year", "week"])
        merged_data = pd.merge(merged_data, df_survey, how="left", on=["snapshot_id", "year", "week"])

        merged_data.to_csv('Linear Regression/Calendar Week/all_data_'+dependent_variable+'.csv', index=False)
        
    merged_data = merged_data.drop(columns=['created_date_sm', 'UpdatedBeginDate_sr', 'UpdatedEndDate_sr'])
    return merged_data

## Run Regression  

In [ ]:
control_variables = ['shipley_vocab', 'shipley_abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism', 'pos_affect', 'neg_affect', 'stai_trait', 'education_level', 'psqi', 'age', 'gender']
sr_life_event_variables = ['Anticipation_sr', 'LifeEventType_sr', 'valence_sr', 'recency_sr', 'status_sr', 'Intimacy_sr', 'Scope_sr', 'significance_sr']
sm_life_event_variables = ['Anticipation_sm', 'LifeEventType_sm', 'valence_sm', 'recency_sm', 'status_sm', 'Intimacy_sm', 'Scope_sm', 'significance_sm']

In [477]:
def get_summary(data, formula):
    model = smf.ols(formula=formula, data = data).fit()

    return model.summary()

In [384]:
data = build_merged_df('stress')

In [400]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['stress'].notnull()]
    
t = get_summary(all_data, 'stress'+'~'+' + '.join(control_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_demo_trait_stress.txt', 'a') as the_file:
    the_file.write(t)

In [401]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['stress'].notnull()]
    
t = get_summary(all_data, 'stress'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_stress.txt', 'a') as the_file:
    the_file.write(t)

In [402]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['stress'].notnull()]
    
t = get_summary(all_data, 'stress'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables + control_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_demo_trait_stress.txt', 'a') as the_file:
    the_file.write(t)

In [403]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['stress'].notnull()]

t = get_summary(all_data, 'stress'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables + control_variables + ['Anticipation_sr:stai_trait', 'Anticipation_sm:stai_trait', 'Scope_sm:openness', 'Scope_sr:openness'])).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_demo_trait_interaction_stress.txt', 'a') as the_file:
    the_file.write(t)

In [406]:
data = build_merged_df('sleep')

In [407]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['sleep'].notnull()]
    
t = get_summary(all_data, 'sleep'+'~'+' + '.join(control_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_demo_trait_sleep.txt', 'a') as the_file:
    the_file.write(t)

In [408]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['sleep'].notnull()]
    
t = get_summary(all_data, 'sleep'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_sleep.txt', 'a') as the_file:
    the_file.write(t)

In [409]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['sleep'].notnull()]
    
t = get_summary(all_data, 'sleep'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables + control_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_demo_trait_sleep.txt', 'a') as the_file:
    the_file.write(t)

In [410]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['sleep'].notnull()]

t = get_summary(all_data, 'sleep'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables + control_variables + ['Anticipation_sr:stai_trait', 'Anticipation_sm:stai_trait', 'Scope_sm:openness', 'Scope_sr:openness'])).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_demo_trait_interaction_sleep.txt', 'a') as the_file:
    the_file.write(t)

In [411]:
data = build_merged_df('anxiety')

In [412]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['anxiety'].notnull()]
    
t = get_summary(all_data, 'anxiety'+'~'+' + '.join(control_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_demo_trait_anxiety.txt', 'a') as the_file:
    the_file.write(t)

In [413]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['anxiety'].notnull()]
    
t = get_summary(all_data, 'anxiety'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_anxiety.txt', 'a') as the_file:
    the_file.write(t)

In [414]:
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['anxiety'].notnull()]
    
t = get_summary(all_data, 'anxiety'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables + control_variables)).as_text()

with open('Linear Regression/Calendar Week/summary/all_sm_sr_demo_trait_anxiety.txt', 'a') as the_file:
    the_file.write(t)

In [554]:
data = build_merged_df('sleep')
all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
all_data = all_data[all_data['sleep'].notnull()]

t = get_summary(all_data, 'sleep'+'~'+' + '.join(sm_life_event_variables + sr_life_event_variables + control_variables + ['agreeableness:Scope_sm','openness:Intimacy_sm:Anticipation_sm']))
t
# with open('Linear Regression/Calendar Week/summary/interaction_sleep_agreeableness_scope_sm_openness_intimacy_sm_openness_anticipation_sm.txt', 'a') as the_file:
#     the_file.write(t)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     4.396
Date:                Tue, 25 Oct 2022   Prob (F-statistic):           2.90e-10
Time:                        09:25:56   Log-Likelihood:                -217.90
No. Observations:                 155   AIC:                             517.8
Df Residuals:                     114   BIC:                             642.6
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                              16.3557     13.475      1.214      0.227     -10.337      43.049
Anticipation_sm[T.Unanticipated]                       -1.8221      1.264     -1.441      0.152      -4.326       0.682
LifeEventType_sm[T.Health]                              2.7769      0.929      2.989      0.003       0.936       4.618
LifeEventType_sm[T.Local]                           -8.156e-15   1.07e-14     -0.764      0.446   -2.93e-14     1.3e-14
LifeEventType_sm[T.Personal]                            2.0281      0.680      2.982      0.004       0.681       3.375
LifeEventType_sm[T.School]                              4.4629      1.051      4.245      0.000       2.380       6.545
LifeEventType_sm[T.Work]                                1.9749      0.841      2.348      0.021       0.308       3.641
status_sm[T.ended]                                      1.6446      0.493      3.333      0.001       0.667       2.622
Anticipation_sr[T.Unanticipated]                        0.9142      0.586      1.559      0.122      -0.248       2.076
LifeEventType_sr[T.Health]                             -1.1999      1.369     -0.877      0.383      -3.911       1.512
LifeEventType_sr[T.Local]                              -1.3136      1.487     -0.884      0.379      -4.259       1.632
LifeEventType_sr[T.Personal]                           -0.5718      1.032     -0.554      0.581      -2.616       1.473
LifeEventType_sr[T.Work]                               -0.5844      1.032     -0.566      0.572      -2.628       1.459
status_sr[T.Ongoing]                                    0.8941      0.947      0.944      0.347      -0.982       2.770
education_level[T.Doctoral]                           3.07e-15   1.43e-15      2.145      0.034    2.35e-16     5.9e-15
education_level[T.Graduate]                             0.6088      0.605      1.007      0.316      -0.589       1.807
education_level[T.High]                                -1.7395      1.185     -1.468      0.145      -4.087       0.608
gender[T.Male]                                          0.0788      0.631      0.125      0.901      -1.172       1.329
valence_sm                                             -0.0057      0.209     -0.027      0.978      -0.420       0.409
recency_sm                                             -0.0012      0.005     -0.253      0.801      -0.011       0.008
Intimacy_sm                                            -7.1669      2.436     -2.942      0.004     -11.992      -2.342
Scope_sm                                                6.0747      3.440      1.766      0.080      -0.739      12.889
significance_sm                                         0.3205      0.563      0.56

In [435]:
def run_regression(data, dependent_variable, variables):

    control_data = data[variables['control']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(control_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    baseline_formula = dependent_variable+'~'+' + '.join(variables['control'])
    baseline = smf.ols(formula=baseline_formula,
                  data = control_data).fit()
    baseline_train = smf.ols(formula=baseline_formula,
                  data = train_data).fit()
    f_obs = baseline_train.predict(test_data).values
    pearson_baseline = scipy.stats.pearsonr(f_obs, f_exp)
    
    
    life_events_data = data[variables['life_events']+[dependent_variable]].dropna()    
    train_data, test_data = train_test_split(life_events_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values

    life_events_formula = dependent_variable+'~'+' + '.join(variables['life_events'])
    life_events = smf.ols(formula=life_events_formula, data = life_events_data).fit() 
    life_events_train = smf.ols(formula=life_events_formula, data = train_data).fit()     
    f_obs = life_events_train.predict(test_data).values
    pearson_life_events = scipy.stats.pearsonr(f_obs, f_exp)    

    
    all_variable_data = data[variables['all_variables']+[dependent_variable]].dropna()
    train_data, test_data = train_test_split(all_variable_data, test_size=0.20, random_state=805)
    f_exp = test_data[dependent_variable].values    
    if len(variables['extra_variables'])>0:
        all_variables_formula = dependent_variable+'~'+' + '.join(variables['all_variables'])+' + '+' + '.join(variables['extra_variables'])
    else:
        all_variables_formula = dependent_variable+'~'+' + '.join(variables['all_variables'])
    all_variables = smf.ols(formula=all_variables_formula, data = all_variable_data).fit() 
    all_variables_train = smf.ols(formula=all_variables_formula, data = train_data).fit() 
    f_obs = all_variables_train.predict(test_data).values
    pearson_all_variables = scipy.stats.pearsonr(f_obs, f_exp)    

#     print("Baseline R-square adj", baseline.rsquared_adj)
#     print("Life Events R-square adj", life_events.rsquared_adj)
#     print("All Variables R-square adj", all_variables.rsquared_adj)
#     print()
#     print(anova_lm(baseline, all_variables))
#     print(anova_lm(life_events, all_variables))
#     print()
#     print("Baseline pearson", pearson_baseline)
#     print("Life Events pearson", pearson_life_events)
#     print("All Variables pearson", pearson_all_variables)    
    return all_variables.rsquared_adj

In [498]:
def run_all_regression(dependent_variable, social_media_v, demographics):
    data = build_merged_df(dependent_variable)

    social_media_data = data[data['LifeEventType_sm'].notnull()]
    social_media_data = social_media_data[social_media_data[dependent_variable].notnull()]

    social_media_variables = {
        'control': control_variables,
        'life_events': sm_life_event_variables,
        'all_variables': sm_life_event_variables + control_variables,
#         'extra_variables':  ['LifeEventType_sm:gender', 'Anticipation_sm:agreeableness']
#         'extra_variables':  [social_media_v+':'+demographics]
    }

    survey_data = data[data['LifeEventType_sr'].notnull()]
    survey_data = survey_data[survey_data[dependent_variable].notnull()]
    survey_variables = {
        'control': control_variables,
        'life_events': sr_life_event_variables,
        'all_variables': sr_life_event_variables + control_variables,
#         'extra_variables':  ['LifeEventType_sr:gender', 'Anticipation_sr:agreeableness']

#         'extra_variables': [survey_v+':'+demographics]
    }
    
    all_data = data[(data['LifeEventType_sr'].notnull()) & (data['LifeEventType_sm'].notnull())]
    all_data = all_data[all_data[dependent_variable].notnull()]
    all_data_variables = {
        'control': control_variables,
        'life_events': sm_life_event_variables + sr_life_event_variables,
        'all_variables': sm_life_event_variables + sr_life_event_variables + control_variables,
#         'extra_variables':  ['LifeEventType_sr:gender', 'Anticipation_sr:agreeableness', 'LifeEventType_sm:gender', 'Anticipation_sm:agreeableness']

        'extra_variables': [social_media_v+':'+demographics]
    }
    
#     print("---------SOCIAL MEDIA---------")
#     run_regression(social_media_data, dependent_variable, social_media_variables)
#     print()
    
#     print("---------SURVEY---------")
#     run_regression(survey_data, dependent_variable, survey_variables)
#     print()
    
#     print("---------SOCIAL MEDIA + SURVEY---------")
    o = run_regression(all_data, dependent_variable, all_data_variables)
#     print()    
    return o

In [542]:
best = []
best_parameter = []
for i in control_variables:
    for k in sm_life_event_variables:
        o = run_all_regression('sleep', k, i)
        best.append(o)
        best_parameter.append((i, k))
    for j in sr_life_event_variables:
        o = run_all_regression('sleep', j, i)
        best.append(o)
        best_parameter.append((i, j))
#             print(i, j, k)

In [543]:
np.array(best)[np.argpartition(best, -20)][-20:]

array([0.41416135, 0.4142864 , 0.41452888, 0.41648794, 0.42148788,
       0.41597769, 0.41711296, 0.4252058 , 0.41711296, 0.41539402,
       0.41735689, 0.42649119, 0.41967958, 0.41739598, 0.422286  ,
       0.42874259, 0.43974222, 0.46090787, 0.44801396, 0.43871047])

In [544]:
np.array(best_parameter)[np.argpartition(best, -20)][-20:]

array([['shipley_vocab', 'status_sr'],
       ['agreeableness', 'Scope_sm'],
       ['psqi', 'Intimacy_sm'],
       ['age', 'Anticipation_sm'],
       ['age', 'LifeEventType_sm'],
       ['stai_trait', 'status_sr'],
       ['shipley_abs', 'status_sr'],
       ['gender', 'recency_sm'],
       ['conscientiousness', 'status_sr'],
       ['extraversion', 'status_sr'],
       ['psqi', 'status_sr'],
       ['psqi', 'valence_sr'],
       ['gender', 'Anticipation_sr'],
       ['openness', 'status_sr'],
       ['education_level', 'significance_sr'],
       ['education_level', 'Scope_sm'],
       ['openness', 'Anticipation_sm'],
       ['agreeableness', 'recency_sm'],
       ['openness', 'Intimacy_sm'],
       ['conscientiousness', 'Intimacy_sm']], dtype='<U17')

In [429]:
run_all_regression('sleep', sm_life_event_variables[j], sr_life_event_variables[j], i)

---------SOCIAL MEDIA---------
All Variables R-square adj 0.13956317751071934

---------SURVEY---------
All Variables R-square adj 0.12482327912831903

---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.4016292570159161



In [423]:
for i in control_variables:
    for j in range(len(sr_life_event_variables)):
        print(i, sr_life_event_variables[j], sm_life_event_variables[j])
        run_all_regression('stress', sm_life_event_variables[j], sr_life_event_variables[j], i)

shipley_vocab Anticipation_sr Anticipation_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.37934278070076144

shipley_vocab LifeEventType_sr LifeEventType_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.37466665520158904

shipley_vocab valence_sr valence_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.38081525965667684

shipley_vocab recency_sr recency_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.3795334835595503

shipley_vocab status_sr status_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.3781725718820944

shipley_vocab Intimacy_sr Intimacy_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.3810709959455385

shipley_vocab Scope_sr Scope_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.40069903491070724

shipley_vocab significance_sr significance_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.38

All Variables R-square adj 0.3848971919076931

neg_affect valence_sr valence_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.40194773946441875

neg_affect recency_sr recency_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.39488646232195557

neg_affect status_sr status_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.3801574973914186

neg_affect Intimacy_sr Intimacy_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.38439600086514414

neg_affect Scope_sr Scope_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.4009260743347658

neg_affect significance_sr significance_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.381239529093926

stai_trait Anticipation_sr Anticipation_sm
---------SOCIAL MEDIA + SURVEY---------
All Variables R-square adj 0.4044562375824998

stai_trait LifeEventType_sr LifeEventType_sm
---------SOCIAL MEDIA + SURVEY---------
All Variab

### Stress 

In [380]:
run_all_regression('stress')

---------SOCIAL MEDIA---------
Baseline R-square adj 0.10907194727705449
Life Events R-square adj 0.06725487817082498
All Variables R-square adj 0.1651083794115682

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0     722.0  340.106908      0.0        NaN       NaN       NaN
1     639.0  292.150119     83.0  47.956789  1.263766  0.066465
   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0     657.0  335.585713      0.0        NaN       NaN           NaN
1     639.0  292.150119     18.0  43.435594  5.277984  1.658511e-11

Baseline pearson (0.26822579146241304, 0.0009805374505623466)
Life Events pearson (0.3063035040283837, 0.00031893328301383327)
All Variables pearson (0.42172200312908076, 3.8789109928713944e-07)

---------SURVEY---------
Baseline R-square adj 0.14474634444517942
Life Events R-square adj 0.02202225569708194
All Variables R-square adj 0.16041843664244015

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0    1253.0  575.8

In [381]:
run_all_regression('sleep')

---------SOCIAL MEDIA---------
Baseline R-square adj 0.13044553100333411
Life Events R-square adj 0.01817114281857879
All Variables R-square adj 0.13634726662465058

   df_resid          ssr  df_diff     ss_diff         F    Pr(>F)
0     653.0  1046.111369      0.0         NaN       NaN       NaN
1     571.0   940.382623     82.0  105.728746  0.782908  0.915876
   df_resid          ssr  df_diff     ss_diff         F        Pr(>F)
0     589.0  1102.758522      0.0         NaN       NaN           NaN
1     571.0   940.382623     18.0  162.375899  5.477477  5.792176e-12

Baseline pearson (0.393488744284707, 2.5579854045044186e-06)
Life Events pearson (0.10033445394993583, 0.2735216332667082)
All Variables pearson (0.34955673467668735, 8.497020646139218e-05)

---------SURVEY---------
Baseline R-square adj 0.08164183887495735
Life Events R-square adj 0.02697651878780316
All Variables R-square adj 0.11798943225585012

   df_resid          ssr  df_diff    ss_diff         F        Pr(>F)
0    

In [382]:
run_all_regression('anxiety')

---------SOCIAL MEDIA---------
Baseline R-square adj 0.1675543210113799
Life Events R-square adj 0.06171937634979263
All Variables R-square adj 0.20629561334806357

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0     722.0  242.774207      0.0        NaN       NaN       NaN
1     639.0  214.403166     83.0  28.371041  1.018748  0.438371
   df_resid         ssr  df_diff    ss_diff         F        Pr(>F)
0     657.0  260.597157      0.0        NaN       NaN           NaN
1     639.0  214.403166     18.0  46.193991  7.648612  2.794435e-18

Baseline pearson (0.35812996941414127, 7.857072126800395e-06)
Life Events pearson (0.19952079170106846, 0.020817849468712943)
All Variables pearson (0.44560120234989004, 6.84109481402126e-08)

---------SURVEY---------
Baseline R-square adj 0.2161289450567555
Life Events R-square adj 0.03614293715079242
All Variables R-square adj 0.23753109378030735

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0    1253.0  447.85924